In [ ]:
import random
import gym
import numpy as np
import collections
from tqdm import tqdm
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
def moving_average(a, window_size):
    cumulative_sum = np.cumsum(np.insert(a, 0, 0)) 
    middle = (cumulative_sum[window_size:] - cumulative_sum[:-window_size]) / window_size
    r = np.arange(1, window_size-1, 2)
    begin = np.cumsum(a[:window_size-1])[::2] / r
    end = (np.cumsum(a[:-window_size:-1])[::2] / r)[::-1]
    return np.concatenate((begin, middle, end))

In [ ]:
class ReplayBuffer:
    ''' 经验回放池 '''
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)  # 队列,先进先出

    def add(self, state, action, reward, next_state, done):  # 将数据加入buffer
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):  # 从buffer中采样数据,数量为batch_size
        transitions = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = zip(*transitions)
        return np.array(state), action, reward, np.array(next_state), done

    def size(self):  # 目前buffer中数据的数量
        return len(self.buffer)

In [ ]:
class Qnet(torch.nn.Module):
    ''' 只有一层隐藏层的Q网络 '''
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Qnet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))  # 隐藏层使用ReLU激活函数
        return self.fc2(x)

In [ ]:
class DQN:
    ''' DQN算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, learning_rate, gamma,
                 epsilon, target_update, device):
        self.action_dim = action_dim
        self.q_net = Qnet(state_dim, hidden_dim,
                          self.action_dim).to(device)  # Q网络
        # 目标网络
        self.target_q_net = Qnet(state_dim, hidden_dim,
                                 self.action_dim).to(device)
        # 使用Adam优化器
        self.optimizer = torch.optim.Adam(self.q_net.parameters(),
                                          lr=learning_rate)
        self.gamma = gamma  # 折扣因子
        self.epsilon = epsilon  # epsilon-贪婪策略
        self.target_update = target_update  # 目标网络更新频率
        self.count = 0  # 计数器,记录更新次数
        self.device = device

    def take_action(self, state):  # epsilon-贪婪策略采取动作
        if np.random.random() < self.epsilon:
            action = np.random.randint(self.action_dim)
        else:
            state = torch.tensor(np.array([state]), dtype=torch.float).to(self.device)
            action = self.q_net(state).argmax().item()
        return action

    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'],
                              dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions']).view(-1, 1).to(
            self.device)
        rewards = torch.tensor(transition_dict['rewards'],
                               dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(transition_dict['next_states'],
                                   dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'],
                             dtype=torch.float).view(-1, 1).to(self.device)

        q_values = self.q_net(states).gather(1, actions)  # Q值
        # 下个状态的最大Q值
        max_next_q_values = self.target_q_net(next_states).max(1)[0].view(-1, 1)
        q_targets = rewards + self.gamma * max_next_q_values * (1 - dones)  # TD误差目标
        dqn_loss = torch.mean(F.mse_loss(q_values, q_targets))  # 均方误差损失函数
        self.optimizer.zero_grad()  # PyTorch中默认梯度会累积,这里需要显式将梯度置为0
        dqn_loss.backward()  # 反向传播更新参数
        self.optimizer.step()

        if self.count % self.target_update == 0:
            self.target_q_net.load_state_dict(
                self.q_net.state_dict())  # 更新目标网络
        self.count += 1

In [ ]:
lr = 2e-3
num_episodes = 250
num_epochs = 10
hidden_dim = 128
gamma = 0.98
epsilon = 0.01
target_update = 50
buffer_size = 10000
minimal_size = 500
batch_size = 128
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")

env_name = 'CartPole-v0'
env = gym.make(env_name)
env.spec.max_episode_steps = 1000
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
replay_buffer = ReplayBuffer(buffer_size)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
agent = DQN(state_dim, hidden_dim, action_dim, lr, gamma, epsilon, target_update, device)

return_list = []
for i in range(num_epochs):
    with tqdm(total=int(num_episodes / num_epochs), desc='Iteration %d' % i) as pbar:
        for i_episode in range(int(num_episodes / num_epochs)):
            episode_return = 0
            state, _ = env.reset()
            done = False
            while not done:
                action = agent.take_action(state)
                next_state, reward, done, _, _ = env.step(action)
                replay_buffer.add(state, action, reward, next_state, done)
                state = next_state
                episode_return += reward
                # 当buffer数据的数量超过一定值后,才进行Q网络训练
                if replay_buffer.size() > minimal_size:
                    b_s, b_a, b_r, b_ns, b_d = replay_buffer.sample(batch_size)
                    transition_dict = {
                        'states': b_s,
                        'actions': b_a,
                        'next_states': b_ns,
                        'rewards': b_r,
                        'dones': b_d
                    }
                    agent.update(transition_dict)
            return_list.append(episode_return)
            if (i_episode + 1) % num_epochs == 0:
                pbar.set_postfix({
                    'episode':
                    '%d' % (num_episodes / num_epochs * i + i_episode + 1),
                    'return':
                    '%.3f' % np.mean(return_list[-num_epochs:])
                })
            pbar.update(1)

In [ ]:
env.spec

In [ ]:
episodes_list = list(range(len(return_list)))
plt.plot(episodes_list, moving_average(return_list, 9))
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('DQN on {}'.format(env_name))
plt.show()

In [ ]:
"""
struct Lot {
    lot_id:      string,
    wafer_type:  int,
    q_time:      int,
    priority:    uint_8,
    wafer_count: int
}
"""
class Lot:
    idx = {i: 9000 for i in range(1, 54)}
    def __init__(self):
        week = random.randint(1, 53)
        while Lot.idx[week] >= 10000:
            week = random.randint(1, 53)
        enable_q_time = bool(random.random() <= 0.1)
        self.lot_id = "BP5" + (("0" + str(week)) if week < 10 else str(week)) + str(Lot.idx[week]) + ".00"
        self.wafer_type = random.randint(1, 4)
        self.qtime = max(0.0, random.normalvariate(100.0, 20.0))
        self.priority = random.randint(0, 7)
        self.wafer_count = int(max(1.0, min(25.0, random.normalvariate(25.0, 15.0))))
        Lot.idx[week] += 1

    def __reset__():
        Lot.idx.clear()

In [ ]:
class Env:
    def __init__(self, reward_threshold: float, max_steps: int, ratio: list[float], rpt: float = 0.0, num_lot: int = 10000):
        self.reward_threshold = reward_threshold
        self.max_steps = max_steps
        self.ratio = ratio
        self.rpt = rpt
        self.num_lot = num_lot
        self.wip_list = [Lot() for i in range(num_lot)]
        self.move = [0, 0, 0, 0]
        self.inventory = [0, 0, 0, 0]
        self.qtime_list = []
        for lot in self.wip_list:
            self.inventory[lot.wafer_type - 1] += 1
            if lot.qtime > 0:
                self.qtime_list.append(lot.qtime) # TODO: How to model the qtime in terms of some data structures